In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [6]:
df =  spark.read.json('people.json')

In [8]:
df.show()


+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



# Type of variables
df.printSchema()  prints the types, but sometimes you need to set the correct type for each column.<BR>
For instance, we want to change age type from long to integer.

In [11]:
df.printSchema()



root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [12]:
from pyspark.sql.types import (StructField, StringType,
                               IntegerType,StructType)

In [16]:
data_schema = [StructField('age',IntegerType(), True),
               StructField('name',StringType(),True)]  
# True: whether or not the filed can be NULL 
#      (i.e., missing values are allowed)

final_struct = StructType(fields = data_schema)

In [17]:
df =  spark.read.json('people.json',schema=final_struct)

In [18]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



<h1>Select vs grab a data</h1>

<h2>Check column</h2>

**df['age']** returns a **column object**, if you want to actually return a DataFrame you need to use **select**

In [22]:
df['age']

Column<b'age'>

In [20]:
df.select('age')

DataFrame[age: int]

In [23]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [24]:
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

In [32]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



<h2>Check Row</h2>

In [28]:
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [29]:
df.head(2)[0]

Row(age=None, name='Michael')

In [30]:
type(df.head(2)[0])

pyspark.sql.types.Row

## Add/rename a column

In [33]:
df.withColumn('double_age',df['age']**2)  # No inplace change

DataFrame[age: int, name: string, double_age: double]

In [44]:
df.withColumnRenamed('age','new_age').show()# No inplace change

+-------+-------+
|new_age|   name|
+-------+-------+
|   null|Michael|
|     30|   Andy|
|     19| Justin|
+-------+-------+



# SQL query support in pyspark
Register the DataFrame as a SQL view


In [38]:
df.createOrReplaceTempView('people')

In [39]:
results = spark.sql("SELECT * FROM people")

In [40]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [45]:
new_result = spark.sql("SELECT * FROM people WHERE age=30")

In [47]:
new_result.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



## Filtering

In [62]:
df = spark.read.csv('appl_stock.csv', inferSchema=True, header=True )
# without inferSchema=True all column types will be considered as string

In [59]:
df.show()


+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [63]:
df.printSchema()


root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



There are two ways for filtering. One using SQL, another way is using python capability.<BR>

Way1: using SQL<BR>
df.filter('close<500').show()    
df.filter('close<500').select(['open','close']).show()

Way2: using python command

In [72]:
df.filter(df['close']<500).select(['open','close']).show()

+------------------+------------------+
|              open|             close|
+------------------+------------------+
|        213.429998|        214.009998|
|        214.599998|        214.379993|
|        214.379993|        210.969995|
|            211.75|            210.58|
|        210.299994|211.98000499999998|
|212.79999700000002|210.11000299999998|
|209.18999499999998|        207.720001|
|        207.870005|        210.650002|
|210.11000299999998|            209.43|
|210.92999500000002|            205.93|
|        208.330002|        215.039995|
|        214.910006|            211.73|
|        212.079994|        208.069996|
|206.78000600000001|            197.75|
|202.51000200000001|        203.070002|
|205.95000100000001|        205.940001|
|        206.849995|        207.880005|
|        204.930004|        199.289995|
|        201.079996|        192.060003|
|192.36999699999998|        194.729998|
+------------------+------------------+
only showing top 20 rows



<h3>Multiple conditions:</h3>

Example:  'close' less than 200 and 'open' larger than 200 <BR>

NB. <BR>
use **&** instead of **and** <BR>
do not forget to separate conditions by **parentheses**
    


In [75]:
df.filter((df['close']<200)&(df['open']>200)).show()

+----------+------------------+----------+----------+----------+---------+------------------+
|      Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+----------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+----------+------------------+----------+----------+----------+---------+------------------+



## collect() and save the result instead of show():
collect() returns the result as **list** of Rows

In [79]:
result = df.filter((df['close']<200)&(df['open']>200)).collect()

In [80]:
type(result)

list

In [81]:
result

[Row(Date='2010-01-22', Open=206.78000600000001, High=207.499996, Low=197.16, Close=197.75, Volume=220441900, Adj Close=25.620401),
 Row(Date='2010-01-28', Open=204.930004, High=205.500004, Low=198.699995, Close=199.289995, Volume=293375600, Adj Close=25.819922000000002),
 Row(Date='2010-01-29', Open=201.079996, High=202.199995, Low=190.250002, Close=192.060003, Volume=311488100, Adj Close=24.883208)]

In [84]:
row = result[0]

<h3>Rows have useful functions:</h3>

In [85]:
row.asDict()

{'Date': '2010-01-22',
 'Open': 206.78000600000001,
 'High': 207.499996,
 'Low': 197.16,
 'Close': 197.75,
 'Volume': 220441900,
 'Adj Close': 25.620401}

In [86]:
row.asDict()['Volume']

220441900

## Groupby and aggregate functions

<h3>Apply a function on groups: </h3>

In [92]:
df = spark.read.csv('sales_info.csv', inferSchema=True, header=True)

In [88]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [89]:
df.groupby('company')

You can run some functions like mean, min, max, count etc. <BR>
NB. They return a DataFrame

In [90]:
df.groupby('company').mean()

DataFrame[company: string, avg(Sales): double]

In [91]:
df.groupby('company').mean().show()

+-------+-----------------+
|company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



<h3>Aggregate through all rows (group is not IMP for us): </h3>

using **.agg()** and pass a **dictionary**

In [94]:
df.agg({'sales':'sum'}).show()

+----------+
|sum(sales)|
+----------+
|    4327.0|
+----------+



<h3> Apply .agg() on groupby : </h3>

In [95]:
group_data = df.groupBy('company')

In [96]:
group_data.agg({'Sales':'max'}).show()

+-------+----------+
|company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



<h3>spark functions</h3>

In [98]:
from pyspark.sql.functions import countDistinct,avg,stddev

In [100]:
df.select(avg('Sales')).show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+

